- 일단 중복 고려 안함. 이동 시 발생하는 reward 0.
- 오직 게임이 끝나고 win, lose, draw에 따라서만 reward 발생
- 원래는 모든 게임트리가 아니라 어느정도 깊이만 탐색함.
- 우선은 전체를 계산하는 다이내믹 프로그래밍 방식으로 구현하겠음.

In [1]:
import time
import os
import pickle
import numpy as np
import pandas as pd
from typing import Tuple
from collections import deque
import copy
from scipy.special import softmax
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 틱택토 환경

In [9]:
class Environment():
    def __init__(self):
        self.n = 3
        self.num_actions = self.n**2
        self.present_state = np.zeros((self.n, self.n))
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win':1, 'lose':-1, 'draw': -0.1, 'good_action':0, 'overlapped':0}
        self.done = False


    def step(self, action_idx:int, agent_turn:bool):
        '''
        에이전트가 선택한 action에 따라 주어지는 next_state, reward, done
        '''
        x, y = np.divmod(action_idx, self.n)
        if agent_turn: # 아님 에이전트 턴과 상대방 턴을 다른 메소드로 나누기
            sign = 1
        else:
            sign = -1

        is_overlap = self.is_overlap(action_idx)

        if is_overlap:
            next_state = self.present_state
            reward = self.reward_dict['overlapped']
            done, is_win = self.is_done(next_state)

        else:
            self.present_state[x,y] = sign
            next_state = self.present_state
            done, is_win = self.is_done(next_state)
            reward = self.reward_dict['good_action']

            if done:
                if is_win == "win":
                    reward = self.reward_dict['win']
                elif is_win == "lose":
                    reward = self.reward_dict['lose']
                else:
                    reward = self.reward_dict['draw']

        self.done = done

        return next_state, reward, done, is_win


    def reset(self):
        '''
        게임판 초기화
        '''
        self.present_state = np.zeros((self.n, self.n))
        self.done = False


    def render(self):
        '''
        print the current state
        '''
        render_state = np.array([['.','.','.'],
                                ['.','.','.'],
                                ['.','.','.']])
        render_str = ""
        for i in range(self.num_actions):
            x, y = np.divmod(i, 3)
            if self.present_state[x,y] == 1:
                render_state[x,y] = 'X'
            elif self.present_state[x,y] == -1:
                render_state[x,y] = 'O'

            render_str += f" {render_state[x,y]}"
            if (i+1) % 3 == 0:
                render_str += "\n" + "-"*11 + "\n"
            else:
                render_str += " |"

        print(render_str)


    def is_overlap(self, action_idx):
        '''
        action이 중복인지 판단하는 함수
        '''
        is_overlap = False
        x, y = np.divmod(action_idx, self.n)
        if self.present_state[x,y] != 0:
            is_overlap = True

        return is_overlap


    def is_done(self, state):
        '''
        틱택토 게임 종료 조건 및 승리 여부 확인하는 함수
        '''
        is_done, is_win = False, "null"

        # 무승부 여부 확인
        if (state==0).sum()==0:
            is_done, is_win = True, "draw"

        # 대각선 빙고 확인
        elif (state[0,0] == state[1,1] == state[2,2]) or (state[0,2] == state[1,1] == state[2,0]):
            if state[1,1] != 0:
                is_done = True

                if state[1,1] == 1: # 승리 조건
                    is_win = "win"
                else: # 패배 조건
                    is_win = "lose"

        else:
            # 가로, 세로 빙고 확인
            for i in range(self.n):
                # 승리 조건
                if state[i,:].sum() == 3 or state[:,i].sum() == 3:
                    is_done, is_win = True, "win"
                    break

                # 패배 조건
                if state[i,:].sum() == -3 or state[:,i].sum() == -3:
                    is_done, is_win = True, "lose"
                    break

        return is_done, is_win

# minimax 알고리즘
- 입력 받은 상태에서 얻을 수 있는 최대값이 뭔지 알려주는 함수
- 개선점: 최대값을 얻을 수 있는 행동이 무엇인지 반환해야한다.
- 현재 모든 경우의 수에 대해 계산하는 minimax 함수이다. 틱택토 정도의 작은 상황에서는 가능하지만, 상태의 수가 많아지면 depth를 도입해 일정 깊이만큼만 탐색하도록 해야한다.

In [33]:
DEPTH = 100
best_action = 0

In [139]:
class Agent():
    def __init__(self, env):
        self.env = env
        self.n = self.env.n
        self.num_actions = self.env.num_actions
        self.actions = torch.tensor(self.num_actions)
        self.best_action = 0


    def minimax(self, present_state, depth, alpha, beta, max_player:bool):
        temp_env = Environment()
        state = copy.deepcopy(present_state)
        temp_env.present_state = state
        done, is_win = temp_env.is_done(state)
        reward = 0

        remain_actions = np.argwhere(state == 0)


        if (done == True) or (depth == 0):
            if is_win == "win":
                reward = temp_env.reward_dict['win']
            elif is_win == "lose":
                reward = temp_env.reward_dict['lose']

            else:
                reward = temp_env.reward_dict['draw']

            return reward

        if max_player:
            maxEval = -np.Inf
            for (x, y) in remain_actions:
                idx = 3*x + y
                child, _, _, _ = temp_env.step(idx, True)

                eval = self.minimax(child, depth-1, alpha, beta, False)

                if maxEval != max(maxEval, eval):
                    self.best_action = idx

                maxEval = max(maxEval, eval)

                alpha = max(alpha, eval)
                if beta <= alpha:
                    break

            return maxEval

        else:
            minEval = np.Inf
            for (x, y) in remain_actions:
                idx = 3*x + y
                child, _, _, _ = temp_env.step(idx, False)

                eval = self.minimax(child, depth-1, alpha, beta, True)
                minEval = min(minEval, eval)

                beta = min(alpha, eval)
                if beta <= alpha:
                    break

            return minEval


    def get_action(self, state, agent_turn):
        self.minimax(state, DEPTH, -np.Inf, np.Inf, True)
        return self.best_action

# 테스트

In [146]:
env = Environment()
agent = Agent(env)
env.render()

 . | . | .
-----------
 . | . | .
-----------
 . | . | .
-----------



In [147]:
action = agent.get_action(env.present_state, True)
next_state, reward, done, is_win = env.step(action, True)
print(action, reward, done, is_win)
env.render()

6 0 False null
 . | . | .
-----------
 . | . | .
-----------
 X | . | .
-----------



In [148]:
action = 4
next_state, reward, done, is_win = env.step(action, False)
print(action, reward, done, is_win)
env.render()

4 0 False null
 . | . | .
-----------
 . | O | .
-----------
 X | . | .
-----------



In [149]:
action = agent.get_action(env.present_state, True)
next_state, reward, done, is_win = env.step(action, True)
print(action, reward, done, is_win)
env.render()

3 0 False null
 . | . | .
-----------
 X | O | .
-----------
 X | . | .
-----------



In [150]:
action = 0
next_state, reward, done, is_win = env.step(action, False)
print(action, reward, done, is_win)
env.render()

0 0 False null
 O | . | .
-----------
 X | O | .
-----------
 X | . | .
-----------



In [151]:
action = agent.get_action(env.present_state, True)
next_state, reward, done, is_win = env.step(action, True)
print(action, reward, done, is_win)
env.render()

8 0 False null
 O | . | .
-----------
 X | O | .
-----------
 X | . | X
-----------



In [152]:
action = 7
next_state, reward, done, is_win = env.step(action, False)
print(action, reward, done, is_win)
env.render()

7 0 False null
 O | . | .
-----------
 X | O | .
-----------
 X | O | X
-----------



In [153]:
action = agent.get_action(env.present_state, True)
next_state, reward, done, is_win = env.step(action, True)
print(action, reward, done, is_win)
env.render()

1 0 False null
 O | X | .
-----------
 X | O | .
-----------
 X | O | X
-----------



In [154]:
action = 2
next_state, reward, done, is_win = env.step(action, False)
print(action, reward, done, is_win)
env.render()

2 0 False null
 O | X | O
-----------
 X | O | .
-----------
 X | O | X
-----------



In [155]:
action = agent.get_action(env.present_state, True)
next_state, reward, done, is_win = env.step(action, True)
print(action, reward, done, is_win)
env.render()

5 -0.1 True draw
 O | X | O
-----------
 X | O | X
-----------
 X | O | X
-----------



## 수동 코딩으로 오류 잡음

In [40]:
temp_env = Environment()
temp_env.present_state = state
done, is_win = temp_env.is_done(state)
is_draw = temp_env.is_draw(state)
reward = 0

remain_actions = np.argwhere(state == 0)

In [45]:
remain_actions

array([[0, 2],
       [1, 0],
       [2, 1]])

In [47]:
x, y = remain_actions[0]
idx = 3*x + y
idx

2

In [50]:
# max turn
maxEval = -np.Inf
max(maxEval, 1)

1

In [51]:
child, _, _, _ = temp_env.step(idx, True)
child

array([[ 1., -1.,  1.],
       [ 0.,  1.,  1.],
       [-1.,  0., -1.]])

In [57]:
state = child

In [59]:
# min turn
temp_env = Environment()
temp_env.present_state = state
done, is_win = temp_env.is_done(state)
is_draw = temp_env.is_draw(state)
reward = 0

remain_actions = np.argwhere(state == 0)

In [60]:
print(done, is_win, is_draw)

False False False


In [64]:
x, y = remain_actions[0]
idx = 3*x + y
idx

3

In [66]:
minEval = np.Inf
min(minEval, 1)

1

In [67]:
child, _, _, _ = temp_env.step(idx, False)
child

array([[ 1., -1.,  1.],
       [-1.,  1.,  1.],
       [-1.,  0., -1.]])

In [78]:
state = child

In [79]:
# max turn
temp_env = Environment()
temp_env.present_state = state
done, is_win = temp_env.is_done(state)
is_draw = temp_env.is_draw(state)
reward = 0

remain_actions = np.argwhere(state == 0)

In [80]:
print(done, is_win, is_draw)

False False False


In [81]:
x, y = remain_actions[0]
idx = 3*x + y
idx

7

In [82]:
maxEval = -np.Inf
child, _, _, _ = temp_env.step(idx, True)
child

array([[ 1., -1.,  1.],
       [-1.,  1.,  1.],
       [-1.,  1., -1.]])

In [83]:
state = child

In [84]:
# 결과값
temp_env = Environment()
temp_env.present_state = state
done, is_win = temp_env.is_done(state)
is_draw = temp_env.is_draw(state)
reward = 0

remain_actions = np.argwhere(state == 0)

print(done, is_win, is_draw)

False False True


- 수동으로!!! 확인해서!!! 어디서 논리오류 난지 알아냄!!!